In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

import torch
from src import models, data, lens, functional
from src.utils import experiment_utils

import logging
from src.utils import logging_utils
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

torch.__version__

'1.13.1'

In [3]:
device = "cuda:0"
mt = models.load_model("mamba-3b", device=device, fp16=False)
# mt = models.load_model("gptj", device=device, fp16=True)

2024-02-01 13:49:45 src.models INFO     loading state-spaces/mamba-2.8b-slimpj (device=cuda:0, fp16=False)
2024-02-01 13:49:45 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2024-02-01 13:49:45 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /state-spaces/mamba-2.8b-slimpj/resolve/main/config.json HTTP/1.1" 200 0
2024-02-01 13:50:02 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /state-spaces/mamba-2.8b-slimpj/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
2024-02-01 13:50:11 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /EleutherAI/gpt-neox-20b/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2024-02-01 13:50:11 src.models INFO     dtype: torch.float32, device: cuda:0, memory: 10.31 GB


In [4]:
dataset = data.load_dataset()

# relation_names = [r.name for r in dataset.relations]
# relation_options = Menu(choices = relation_names, value = relation_names)
# show(relation_options) # !caution: tested in a juputer-notebook. baukit visualizations are not supported in vscode.

2024-02-01 13:50:11 src.data DEBUG    no paths provided, using default data dir: /home/local_arnab/Codes/relations/notebooks/../data
2024-02-01 13:50:11 src.data DEBUG    /home/local_arnab/Codes/relations/notebooks/../data is directory, globbing for json files...
2024-02-01 13:50:11 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/characteristic_gender.json
2024-02-01 13:50:11 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/degree_gender.json
2024-02-01 13:50:11 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_birthplace.json
2024-02-01 13:50:11 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_gender.json
2024-02-01 13:50:11 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_religion.json
2024-02-01 13:50:11 src.data DEBUG    found relation 

2024-02-01 13:50:11 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/commonsense/object_superclass.json
2024-02-01 13:50:11 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/commonsense/substance_phase.json
2024-02-01 13:50:11 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/commonsense/task_done_by_person.json
2024-02-01 13:50:11 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/commonsense/task_done_by_tool.json
2024-02-01 13:50:11 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/commonsense/word_sentiment.json
2024-02-01 13:50:11 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/commonsense/work_location.json
2024-02-01 13:50:11 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/factual/city_in_coun

In [5]:
relation_name = "country capital city"
relation = dataset.filter(relation_names=[relation_name])[0]
print(f"{relation.name} -- {len(relation.samples)} samples")
print("------------------------------------------------------")

experiment_utils.set_seed(12345) # set seed to a constant value for sampling consistency
train, test = relation.split(5)
print("\n".join([sample.__str__() for sample in train.samples]))

2024-02-01 13:50:11 src.data DEBUG    filtering to only relations: ['country capital city']
country capital city -- 24 samples
------------------------------------------------------
2024-02-01 13:50:11 src.utils.experiment_utils INFO     setting all seeds to 12345
China -> Beijing
Japan -> Tokyo
Italy -> Rome
Brazil -> Bras\u00edlia
Turkey -> Ankara


In [6]:
################### hparams ###################
layer = 20
beta = 8
###############################################

In [7]:
len(train.samples)

5

In [8]:
hasattr(mt.model, "backbone")

True

In [9]:
models.determine_layer_paths(mt)[layer]

'backbone.layers.20'

In [10]:
prompt = mt.tokenizer.eos_token + " Michael Jordan professionally played the sport of"

tokenized = mt.tokenizer(prompt, return_tensors="pt").to(device)

In [11]:
models.determine_final_layer_norm_path(mt)

'backbone.norm_f'

In [12]:
import baukit

layer_out = models.determine_layer_paths(mt)[layer]
mixer_out = models.determine_layer_paths(mt)[layer] + ".mixer"
layer_in = models.determine_layer_paths(mt)[layer+1]
final_layer = models.determine_layer_paths(mt)[-1]
final_mixer = models.determine_layer_paths(mt)[-1] + ".mixer"

with baukit.TraceDict(
    module=mt.model,
    layers=[
        layer_out,
        mixer_out,
        layer_in,
        final_mixer,
        final_layer,
        "backbone"
    ],
    retain_input=True,
) as traces:
    output = mt(**tokenized)

In [13]:
baukit.get_module(mt.model, "backbone.norm_f")

RMSNorm()

In [14]:
traces["backbone"].output

tensor([[[-2.8400e-01,  3.3097e-01,  2.6135e-01,  ...,  3.9824e-01,
          -3.1995e-01, -1.3488e-01],
         [ 6.9976e-02,  1.3667e-01, -1.0784e-02,  ...,  4.0056e-01,
          -5.1812e-01,  4.9508e-01],
         [-4.9874e-02, -6.2232e-03, -1.4993e-01,  ..., -2.2583e-01,
           6.6746e-04, -7.6109e-02],
         ...,
         [ 1.2778e+00,  4.1307e-01, -3.9810e-01,  ..., -7.6271e-01,
          -5.7157e-01, -9.6000e-01],
         [-4.3011e-01, -6.2957e-01, -6.3809e-01,  ..., -1.0444e+00,
          -9.1710e-01, -1.6055e-01],
         [ 1.8030e-01, -4.9389e-01, -7.1039e-01,  ..., -6.1373e-01,
          -3.5799e-01, -3.7761e-01]]], device='cuda:0',
       grad_fn=<LayerNormFnBackward>)

In [15]:
traces[final_layer].output

(tensor([[[ 0.5940,  1.9729, -0.7331,  ...,  2.2618, -2.4105,  0.6027],
          [ 1.2616,  0.6781, -0.8456,  ...,  1.3468, -2.0678,  3.0296],
          [-0.0950,  1.0767, -0.0086,  ..., -0.8217,  0.7617,  0.6352],
          ...,
          [ 7.5944,  2.9077, -2.3740,  ..., -2.0662,  0.9440, -1.2279],
          [-7.4484, -2.3972, -2.6358,  ..., -4.6405, -0.2366,  1.1990],
          [-2.2326, -0.7476, -2.6844,  ..., -3.6317,  1.6523, -0.6629]]],
        device='cuda:0', grad_fn=<MambaInnerFnBackward>),
 tensor([[[-2.9466,  0.8577,  3.0205,  ...,  0.9754, -0.4319, -1.6834],
          [-0.6205,  0.6147,  0.7412,  ...,  2.2544, -3.0231,  1.3576],
          [-0.1985, -1.1145, -0.9235,  ..., -0.4823, -0.7575, -1.0683],
          ...,
          [ 4.5634,  1.1499, -1.6278,  ..., -5.0547, -6.7761, -7.6065],
          [ 3.3821, -3.7479, -3.7378,  ..., -5.0487, -9.0620, -2.6670],
          [ 3.8406, -3.8000, -4.0094,  ..., -1.7394, -5.0764, -2.5944]]],
        device='cuda:0', grad_fn=<LayerNormF

In [16]:
from torch import nn

class RMSNorm(nn.Module):
    def __init__(self,
                 d_model: int,
                 eps: float = 1e-5):
        super().__init__()
        self.eps = eps
        self.weight = baukit.get_module(mt.model, "backbone.norm_f").weight


    def forward(self, x):
        output = x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps) * self.weight

        return output

custom_rms = RMSNorm(d_model = models.determine_hidden_size(mt))
block_output, residual = traces[final_layer].output
backbone_output = custom_rms(block_output + residual)

backbone_output

tensor([[[-2.8400e-01,  3.3097e-01,  2.6135e-01,  ...,  3.9824e-01,
          -3.1995e-01, -1.3488e-01],
         [ 6.9976e-02,  1.3667e-01, -1.0784e-02,  ...,  4.0055e-01,
          -5.1812e-01,  4.9508e-01],
         [-4.9874e-02, -6.2232e-03, -1.4993e-01,  ..., -2.2583e-01,
           6.6746e-04, -7.6109e-02],
         ...,
         [ 1.2778e+00,  4.1307e-01, -3.9810e-01,  ..., -7.6271e-01,
          -5.7157e-01, -9.6000e-01],
         [-4.3011e-01, -6.2957e-01, -6.3809e-01,  ..., -1.0444e+00,
          -9.1710e-01, -1.6055e-01],
         [ 1.8030e-01, -4.9389e-01, -7.1039e-01,  ..., -6.1373e-01,
          -3.5799e-01, -3.7761e-01]]], device='cuda:0', grad_fn=<MulBackward0>)

In [17]:
torch.allclose(traces[mixer_out].output, traces[layer_out].output[0])

True

In [18]:
baukit.get_module(mt.model, "backbone.norm_f").bias

In [19]:
hasattr(mt.model, "backbone")

True

In [20]:
from src.operators import JacobianIclMeanEstimator
experiment_utils.set_seed(12345)

estimator = JacobianIclMeanEstimator(
    mt = mt, 
    h_layer = layer,
    beta = beta
)
operator = estimator(
    relation.set(
        samples=train.samples, 
    )
)

2024-02-01 13:50:13 src.utils.experiment_utils INFO     setting all seeds to 12345
2024-02-01 13:50:13 src.operators WARNING  relation has > 1 prompt_templates, will use first (The capital city of {} is)
2024-02-01 13:50:13 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The capital city of Brazil is Bras\u00edlia
The capital city of Italy is Rome
The capital city of China is
2024-02-01 13:50:13 src.operators DEBUG    note that subject=China, h_index=26
h.shape=torch.Size([2560]) | h.requires_grad=True
Calculating Jacobians ...


  0%|          | 0/2560 [00:00<?, ?it/s]

2024-02-01 13:57:11 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The capital city of China is Beijing
The capital city of Brazil is Bras\u00edlia
The capital city of Japan is
2024-02-01 13:57:11 src.operators DEBUG    note that subject=Japan, h_index=26
h.shape=torch.Size([2560]) | h.requires_grad=True
Calculating Jacobians ...


  0%|          | 0/2560 [00:00<?, ?it/s]

2024-02-01 14:04:01 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The capital city of Brazil is Bras\u00edlia
The capital city of China is Beijing
The capital city of Italy is
2024-02-01 14:04:01 src.operators DEBUG    note that subject=Italy, h_index=26
h.shape=torch.Size([2560]) | h.requires_grad=True
Calculating Jacobians ...


  0%|          | 0/2560 [00:00<?, ?it/s]

2024-02-01 14:10:50 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The capital city of Italy is Rome
The capital city of China is Beijing
The capital city of Brazil is
2024-02-01 14:10:50 src.operators DEBUG    note that subject=Brazil, h_index=21
h.shape=torch.Size([2560]) | h.requires_grad=True
Calculating Jacobians ...


  0%|          | 0/2560 [00:00<?, ?it/s]

2024-02-01 14:17:41 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is
2024-02-01 14:17:41 src.operators DEBUG    note that subject=Turkey, h_index=26
h.shape=torch.Size([2560]) | h.requires_grad=True
Calculating Jacobians ...


  0%|          | 0/2560 [00:00<?, ?it/s]

In [21]:
operator.metadata

{'Jh': [tensor([ 0.0294,  0.0021,  0.1181,  ..., -0.0475,  0.1220, -0.1429]),
  tensor([ 0.1958, -0.0681,  0.1216,  ..., -0.1533,  0.1851,  0.0289]),
  tensor([ 0.0258,  0.0113,  0.0721,  ...,  0.0790,  0.1656, -0.2051]),
  tensor([-0.1199,  0.1173,  0.0563,  ..., -0.0231,  0.0688, -0.1298]),
  tensor([ 0.2618, -0.0363, -0.0281,  ...,  0.0333, -0.2836, -0.1635])],
 '|w|': [69.82915496826172,
  71.52497100830078,
  60.89453125,
  83.10833740234375,
  88.31616973876953],
 '|b|': [245.52316284179688,
  252.23367309570312,
  240.4381103515625,
  281.2376403808594,
  278.8184509277344]}

In [22]:
from src.lens import logit_lens
from src import models

logit_lens(mt = mt, h = operator.metadata["Jh"][0].to(models.determine_device(mt)) + operator.bias)

([(' ', 46.49),
  ('\n', 44.75),
  (' (', 43.876),
  (' -', 43.401),
  ('.', 42.901),
  (' \\', 42.75),
  ('?', 42.646),
  (' Be', 42.488),
  ('...', 42.423),
  (' S', 42.323)],
 {})

In [23]:
from src.functional import predict_next_token

predict_next_token(
    mt = mt, 
    # prompt = mt.tokenizer.eos_token + " The capital of {} is".format("France"),
    prompt = mt.tokenizer.eos_token + " The superlative of {} is".format("good"),
)

[[PredictedToken(token=' "', prob=0.09106151759624481),
  PredictedToken(token=' not', prob=0.08697706460952759),
  PredictedToken(token=' good', prob=0.071841299533844),
  PredictedToken(token=' the', prob=0.06490865349769592),
  PredictedToken(token=' better', prob=0.06266635656356812)]]

# Checking $faithfulness$

In [24]:
test = functional.filter_relation_samples_based_on_provided_fewshots(
    mt=mt, test_relation=test, prompt_template=operator.prompt_template, batch_size=4
)

2024-02-01 14:24:30 src.functional DEBUG    filtering for knowns using prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of {} is"


2024-02-01 14:24:30 src.functional DEBUG    sample.subject='South Korea', sample.object='Seoul', predicted=' Seoul' (p=0.960), known=(✓)
2024-02-01 14:24:30 src.functional DEBUG    sample.subject='Colombia', sample.object='Bogot\\u00e1', predicted=' Bog' (p=0.971), known=(✓)
2024-02-01 14:24:30 src.functional DEBUG    sample.subject='Saudi Arabia', sample.object='Riyadh', predicted=' R' (p=0.739), known=(✓)
2024-02-01 14:24:30 src.functional DEBUG    sample.subject='France', sample.object='Paris', predicted=' Paris' (p=0.984), known=(✓)
2024-02-01 14:24:30 src.functional DEBUG    sample.subject='Mexico', sample.object='Mexico City', predicted=' Mexico' (p=0.956), known=(✓)
2024-02-01 14:24:30 src.functional DEBUG    sample.subject='Pakistan', sample.object='Islamabad', predicted=' Islam' (p=0.924), known=(✓)
2024-02-01 14:24:30 src.functional DEBUG    sample.subject='Argentina', sample.object='Buenos Aires', predicted=' Buenos' (p=0.940), known=(✓)
2024-02-01 14:24:30 src.functional DE

In [36]:
sample = [s for s in test.samples if s.subject == "Canada"][0]
print(sample)
operator(subject = sample.subject).predictions

Canada -> Ottawa
2024-02-01 17:54:42 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of Canada is"


[PredictedToken(token=' Ottawa', prob=0.39848828315734863),
 PredictedToken(token=' Washington', prob=0.1733689308166504),
 PredictedToken(token=' ', prob=0.15794815123081207),
 PredictedToken(token=' Tokyo', prob=0.08120059967041016),
 PredictedToken(token=' Capital', prob=0.0415043868124485)]

In [37]:
hs_and_zs = functional.compute_hs_and_zs(
    mt = mt,
    prompt_template = operator.prompt_template,
    subjects = [sample.subject],
    h_layer= operator.h_layer,
)
h = hs_and_zs.h_by_subj[sample.subject]

## Approximating LM computation $F$ as an affine transformation

### $$ F(\mathbf{s}, c_r) \approx \beta \, W_r \mathbf{s} + b_r $$

In [38]:
z = 5 * (operator.weight @ h) + operator.bias

lens.logit_lens(
    mt = mt,
    h = z,
    get_proba = True
)

([(' ', 0.469),
  (' Washington', 0.094),
  (' Tokyo', 0.075),
  ('\n', 0.072),
  (' Capital', 0.051),
  (' -', 0.027),
  (' capital', 0.025),
  (' Ottawa', 0.023),
  (' the', 0.019),
  (' (', 0.017)],
 {})

In [39]:
correct = 0
wrong = 0
for sample in test.samples:
    predictions = operator(subject = sample.subject).predictions
    known_flag = functional.is_nontrivial_prefix(
        prediction=predictions[0].token, target=sample.object
    )
    print(f"{sample.subject=}, {sample.object=}, ", end="")
    print(f'predicted="{functional.format_whitespace(predictions[0].token)}", (p={predictions[0].prob}), known=({functional.get_tick_marker(known_flag)})')
    
    correct += known_flag
    wrong += not known_flag
    
faithfulness = correct/(correct + wrong)

print("------------------------------------------------------------")
print(f"Faithfulness (@1) = {faithfulness}")
print("------------------------------------------------------------")

2024-02-01 17:54:46 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of Argentina is"


sample.subject='Argentina', sample.object='Buenos Aires', predicted=" Buenos", (p=0.9405075907707214), known=(✓)
2024-02-01 17:54:46 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of Australia is"
sample.subject='Australia', sample.object='Canberra', predicted=" Sydney", (p=0.7758105397224426), known=(✗)
2024-02-01 17:54:46 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of Canada is"
sample.subject='Canada', sample.object='Ottawa', predicted=" Ottawa", (p=0.39848828315734863), known=(✓)
2024-02-01 17:54:46 src.operators DEBUG    computing h from prompt "<|

# $causality$

In [40]:
################### hparams ###################
rank = 100
###############################################

In [41]:
experiment_utils.set_seed(12345) # set seed to a constant value for sampling consistency
test_targets = functional.random_edit_targets(test.samples)

2024-02-01 17:54:49 src.utils.experiment_utils INFO     setting all seeds to 12345


## setup

In [42]:
source = test.samples[0]
target = test_targets[source]

f"Changing the mapping ({source}) to ({source.subject} -> {target.object})"

'Changing the mapping (Argentina -> Buenos Aires) to (Argentina -> Riyadh)'

### Calculate $\Delta \mathbf{s}$ such that $\mathbf{s} + \Delta \mathbf{s} \approx \mathbf{s}'$

<p align="center">
    <img align="center" src="../causality_crop" style="width:80%;"/>
</p>

Under the relation $r =\, $*plays the instrument*, and given the subject $s =\, $*Miles Davis*, the model will predict $o =\, $*trumpet* **(a)**; and given the subject $s' =\, $*Cat Stevens*, the model will now predict $o' =\, $*guiter* **(b)**. 

If the computation from $\mathbf{s}$ to $\mathbf{o}$ is well-approximated by $operator$ parameterized by $W_r$ and $b_r$ **(c)**, then $\Delta{\mathbf{s}}$ **(d)** should tell us the direction of change from $\mathbf{s}$ to $\mathbf{s}'$. Thus, $\tilde{\mathbf{s}}=\mathbf{s}+\Delta\mathbf{s}$ would be an approximation of $\mathbf{s}'$ and patching $\tilde{\mathbf{s}}$ in place of $\mathbf{s}$ should change the prediction to $o'$ = *guitar* 

In [43]:
def get_delta_s(
    operator, 
    source_subject, 
    target_subject,
    rank = 100,
    fix_latent_norm = None, # if set, will fix the norms of z_source and z_target
):
    w_p_inv = functional.low_rank_pinv(
        matrix = operator.weight,
        rank=rank,
    )
    hs_and_zs = functional.compute_hs_and_zs(
        mt = mt,
        prompt_template = operator.prompt_template,
        subjects = [source_subject, target_subject],
        h_layer= operator.h_layer,
        z_layer=-1,
    )

    z_source = hs_and_zs.z_by_subj[source_subject]
    z_target = hs_and_zs.z_by_subj[target_subject]
    
    z_source *= fix_latent_norm / z_source.norm() if fix_latent_norm is not None else 1.0
    z_target *= z_source.norm() / z_target.norm() if fix_latent_norm is not None else 1.0

    delta_s = w_p_inv @  (z_target.squeeze() - z_source.squeeze())

    return delta_s, hs_and_zs

delta_s, hs_and_zs = get_delta_s(
    operator = operator,
    source_subject = source.subject,
    target_subject = target.subject,
    rank = rank
)

In [44]:
import baukit

def get_intervention(h, int_layer, subj_idx):
    def edit_output(output, layer):
        if(layer != int_layer):
            return output
        functional.untuple(output)[:, subj_idx] = h 
        return output
    return edit_output

prompt = operator.prompt_template.format(source.subject)

h_index, inputs = functional.find_subject_token_index(
    mt=mt,
    prompt=prompt,
    subject=source.subject,
)

h_layer, z_layer = models.determine_layer_paths(model = mt, layers = [layer, -1])

with baukit.TraceDict(
    mt.model, layers = [h_layer, z_layer],
    edit_output=get_intervention(
#         h = hs_and_zs.h_by_subj[source.subject],         # let the computation proceed as usual
        h = hs_and_zs.h_by_subj[source.subject] + delta_s, # replace s with s + delta_s
        int_layer = h_layer, 
        subj_idx = h_index
    )
) as traces:
    outputs = mt(
        input_ids = inputs.input_ids,
        attention_mask = inputs.attention_mask,
    )

logits = outputs.logits[0][-1] if hasattr(outputs, "logits") else outputs[0][-1]
lens.interpret_logits(
    mt = mt, 
    logits = logits, 
    get_proba=True
)

[(' R', 0.661),
 (' Me', 0.106),
 (' Jed', 0.029),
 (' Saudi', 0.027),
 (' Ri', 0.022),
 ('\n', 0.01),
 (' Dh', 0.007),
 (' Mak', 0.006),
 (' Med', 0.005),
 (' J', 0.004)]

## Measuring causality

In [45]:
from src.editors import LowRankPInvEditor

svd = torch.svd(operator.weight.float())
editor = LowRankPInvEditor(
    lre=operator,
    rank=rank,
    svd=svd,
)

In [46]:
# precomputing latents to speed things up
hs_and_zs = functional.compute_hs_and_zs(
    mt = mt,
    prompt_template = operator.prompt_template,
    subjects = [sample.subject for sample in test.samples],
    h_layer= operator.h_layer,
    z_layer=-1,
    batch_size = 2
)

success = 0
fails = 0

for sample in test.samples:
    target = test_targets.get(sample)
    assert target is not None
    edit_result = editor(
        subject = sample.subject,
        target = target.subject
    )
    
    success_flag = functional.is_nontrivial_prefix(
        prediction=edit_result.predicted_tokens[0].token, target=target.object
    )
    
    print(f"Mapping {sample.subject} -> {target.object} | edit result={edit_result.predicted_tokens[0]} | success=({functional.get_tick_marker(success_flag)})")
    
    success += success_flag
    fails += not success_flag
    
causality = success / (success + fails)

print("------------------------------------------------------------")
print(f"Causality (@1) = {causality}")
print("------------------------------------------------------------")

2024-02-01 17:54:54 src.editors DEBUG    computing low-rank pinv (rel=<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of {} is)
Mapping Argentina -> Riyadh | edit result=' R' (p=0.521) | success=(✓)
Mapping Australia -> Buenos Aires | edit result=' Buenos' (p=0.885) | success=(✓)
Mapping Canada -> Abuja | edit result=' Lag' (p=0.632) | success=(✗)
Mapping Chile -> Lima | edit result=' L' (p=0.926) | success=(✓)
Mapping Colombia -> Berlin | edit result=' Berlin' (p=0.852) | success=(✓)
Mapping Egypt -> Mexico City | edit result=' Mexico' (p=0.955) | success=(✓)
Mapping France -> Riyadh | edit result=' Me' (p=0.514) | success=(✗)
Mapping Germany -> Cairo | edit result=' Cairo' (p=0.923) | success=(✓)
Mapping India -> Lima | edit result=' L' (p=0.928) | success=(✓)
Mapping Mexico -> Santiago | edit result=' Santiago' (p=0